<div class="alert alert-danger">

**Refresh your knowledge of the provided classes *BayesNet* and *Variable* by reading the `BayesNet Introduction` notebook** before you start working on this problem set! 
    
</div>


# Structure Learning
## Problem Set 4 

## Probabilistic Models UE

---

In the fourth assignment, you will implement methods that learn the *structure* and *parameters* of a Bayesian network using *score-based search*. In particular, you will implement:

* the BIC-Score as an alternative to the log-likelihood-Score
* complete search in space restricted by a pre-determined variable order
* greedy local search in the full search space

You will then use these functions to automatically find graph structures for the stroke model from Problem Set 3.

## Submission

**Due-Date:** see Moodle
 
**Automatic Grading:** 

- Replace the placeholders `# YOUR CODE HERE` `raise NotImplementedError()` / `YOUR ANSWER HERE` with your code / answers.
- Put results in the corresponding variable; otherwise, we will not grade your solution (i.e., we assign 0 points).
- Do not delete or add cells.

<div class="alert alert-warning">
    
**Submission:** As a ZIP-package via Moodle; the ZIP-package **must have the following structure**:
    
    <student ID, (k + 8 digits), e.g. k01234567>.zip
    +-- Problem_1.ipynb
    |-- Supplementary Materials (optional)
    + 

**Questions?** Post them into the Problem Set Forum!        
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Iterator, List, Optional
from bayesian_network import BayesNet, Variable
from utils import maximum_likelihood_estimate, log_likelihood, get_default_bayes_net
from utils import sample_forward, draw_graph, evaluate_bayes_net, compare_bayes_net

#### General Remarks

The following is a quick refresher on how to use the BayesNet class, including an introduction to visually depicting a Bayesian Network structure from a BayesNet object. Assume we want to encode the model that factorizes in the following way:

$$ P(A, B, C) = P(A) \cdot P(B \mid A) \cdot P(C \mid A, B).$$

Variables $A, B$ and $C$ will get indices 0, 1, and 2, respectively. The corresponding list will then look like this (following the same naming conventions as used in the previous exercises):

In [ ]:
_A_, _B_, _C_ = 0, 1, 2
# create dummy PDTs
pdts = [
    (np.ones((2,))*0.5, (_A_,)), 
    (np.ones((2,2))*0.5, (_B_, _A_)), 
    (np.ones((2,2,2))*0.5, (_C_, _A_, _B_))
]

Each tuple in the list corresponds to a variable, where
 - the first element in each tuple contains the (conditional) PDT, 
 - the second one describes the table's semantics (indices).

A new BayesNet can be created by passing a tuple for each variable to the constructor. Use the asterisk (\*) operator to pass an arbitrary number of tuples as arguments to the constructor:

In [ ]:
bn = BayesNet(*pdts)

You can access the PDTs and the corresponding indices of a *BayesNet* object via the member variables `pdts` and `indices`, respectively:

In [ ]:
print(bn.pdts)
print(bn.indices)

Note that they are returned as tuples to make sure they should not be modified. With `list(bn.pdts)` and `list(bn.indices)` you can convert them back to lists if needed.

We are also providing a new utility function to draw the structure of a BayesNet:

In [ ]:
draw_graph(
    bn, 
    node_names=['A', 'B', 'C'], 
    pos={'A': [0., 1.], 'B': [1., 1.], 'C': [.5, .5]}
)

In [ ]:
help(draw_graph)

### Bayes Information Criterion

<div class="alert alert-warning">
    Implement the <i>bic_score</i> function, which computes the Bayes Information Criterion Score (BIC-Score) for a network graph structure under the given data, assuming a Dirichlet prior over the model parameters. (2 points)
</div>

`bic_score` takes two parameters:
- `data` is a NumPy array of shape `(num_samples, num_variables)`.
- `bayes_net` is a BayesNet object, representing the graph structure for which to compute the BIC-Score. *Assume that the parameters of this bayes_net are set to the maximum likelihood estimate w.r.t. to the data*.

`bic_score` must return one object:
- The BIC-Score, a floating-point number < 0.


*Hint*:
- **You *may* assume that all variables are boolean.**
- Use the formula from the lecture slides (slide deck 5b, pp. 48-51)!
- The dimension of a Bayes net is the total number of independent (non-redundant) parameters in the Bayes net.
- Use the natural logarithm for your computations, i.e. `np.log`.
- You can use `Variable.num_values` to fetch the cardinality of a Variable. This is especially helpful for a general solution, where Variables can have more than 2 values. 
- Use our implementation of `log_likelihood` (see below) to compute the log-likelihood of the model relative to the data.

In [ ]:
help(log_likelihood)

In [ ]:
def bic_score(data: np.ndarray, bayes_net: BayesNet) -> float:
    """
    Computes the Bayesian information criterion score assuming a Dirichlet prior over the model parameters.
    :param data:      data to compute the bic score of, a np.ndarray of shape (num_samples, len(bayes_net)).
    :param bayes_net: the model, a BayesNet object.
    :return:          BIC score
    """
    
    assert data.shape[1] == len(bayes_net)
    
    bic = 0
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return bic

In [ ]:
# sanity check
bayes_net = get_default_bayes_net()
data = np.array([[0,0,0,0,0], [1,1,1,1,1], [1,1,0,0,0]], dtype=np.int64)

bic = bic_score(data, bayes_net)
assert isinstance(bic,float), f'`bic_score` should return type float but returns {type(bic)}'
assert -16 < bic < -15, f'BIC needs to be in ]-16, -15[, but got {bic}'



## Complete Search in Restricted Space

In the first part, we will focus on complete search in a restricted space.

<div class="alert alert-warning">
Implement <i>complete_search</i>, a function that learns the structure and parameters of a Bayesian net using score-based search over a search space restricted by a pre-determined variable order. (3 points)
</div>

`complete_search` takes three parameters:
- `data` is a NumPy array of shape `(num_samples, num_variables)`.
- `variable_order` is a list of variable ids (ints) specifying the topological order.
- `score_fun` is the function used to score the candidate graph structure — defaults to `bic_score`. Higher score means better model.
- `max_parents` is an integer value specifying the maximum number of parents a variable can have - defaults to `None`, meaning no restriction

`complete_search` must return one object:
- a BayesNet object, which represents the model with the highest score under the given topological order.

*Hints:*
- Get familiar with the algorithm depicted in the lecture slides (slide deck 5b, pp. 54-56)
- Use `score_fun` to compute the score. Do not directly call your implementation of `bic_score`.
- You may use [`itertools.combinations`](https://docs.python.org/3/library/itertools.html#itertools.combinations) to get combinations of parent variables. For the N-th variable in `variable_order`, there are exactly $2^{N-1}$ possible parent sets to be considered (in case `max_parents` is set to `None`, otherwise the number of possible sets may be reduced).
- [`itertools.chain.from_iterable`](https://docs.python.org/3/library/itertools.html#itertools.chain.from_iterable) might become handy for flattening lists of lists of variable indexes.
- Mind that python-objects, such as lists, are passed by reference; updating a value in the list will affect all objects holding the reference! To avoid unintended change, create a copy of the list before you modify it:
```python
best_pdts.copy()
```
- Use a BayesNet only consisting of independent Variables (e.g., no connections) as starting point. Iteratively compare it with other ones and always keep the best.
- Use the provided implementation of `maximum_likelihood_estimate` to estimate the parameters of a table:

In [ ]:
help(maximum_likelihood_estimate)

In [ ]:
import itertools as it
import copy
def complete_search(data: np.ndarray, variable_order: list, score_fun=bic_score, max_parents: Optional[int] = None) -> BayesNet:
    """
    Learns the structure and parameters of a Bayesian net using score-based search over a 
        search space restricted by a pre-determined variable order.
    :param data: the training data, a NumPy array of shape (num_samples, len(bayes_net)).
    :param variable_order: a list containing the topological order of the variables (list of indices).
    :param score_fun: is the function used to score the candidate graph structure — defaults to `bic_score`. 
    :param max_parents: an integer value specifying the maximum number of parents a variable can have — defaults to `None` (no restriction). 
    :return: a BayesNet object, which represents the model with the highest BIC-Score under the given topological order.
    """
    assert len(variable_order) == data.shape[1]
    assert max_parents is None or max_parents >= 0
    
    best_pdts = [maximum_likelihood_estimate(data, variable_id) for variable_id in variable_order]
    best_indices = [(variable_id,) for variable_id in variable_order]
    # you can create a BayesNet from these descriptors with:
    best_model = BayesNet(*zip(best_pdts, best_indices))
    
    # YOUR CODE HERE
    raise NotImplementedError()
    return best_model

In [ ]:
data = np.array([[0,0,0,0,1],[0,0,0,1,0],[0,1,0,1,0],[1,0,0,1,0],[0,1,0,0,1],[1,0,0,0,1]]*10, dtype=np.int64)
bayes_net = complete_search(data, [0,1,2,3,4])
assert len(bayes_net) == 5, f'\nBayesNet should contain all 5 Variables.' \
                            f'\nIt only contained {len(bayes_net)} being {set(it.chain(*bayes_net.indices))}.'\
                            f'\nMissing: {set(range(5))-set(it.chain(*bayes_net.indices))}.'\
                            f'\nMost likely there is an error in how you construct your `new` BayesNet during learning.'

assert np.all(np.isclose(bayes_net.pdts[0], np.array([0.66, 0.33]), atol=0.01))
assert bayes_net.indices[1] == (1,0), 'Algorithm should favor connection B_A'

bayes_net = complete_search(data, [0,1,2,3,4], score_fun=lambda d, n: 0.0)
assert bayes_net.indices[1] == (1,), 'it seems complete_search() uses bic_score() instead of score_fun()'

data = np.random.choice([0,1], (10,5)).astype(np.int64)
max_parents = 2
bayes_net = complete_search(data, [0,1,2,3,4], score_fun=log_likelihood, max_parents=max_parents)
assert max([len(var.parents) for var in bayes_net]) == max_parents, 'it seems that the parameter `max_parents` is not correctly used'


## Heuristic Search in Full Space

In the second part, you will implement heuristic search in the full search space. In contrast to *Complete Search in Restricted Space*, we do not restrict the search space, but we can not afford to search the complete space. Instead, our search algorithm will only visit a small part of the full search space, guided by a heuristic function.   

In the following, you will implement the five functions:

- `is_acyclic(bn: BayesNet) -> bool`: checks if a Bayesian Network is acyclic
- `link_addition(data: np.array, bn: BayesNet) -> Iterator[BayesNet]`: search operator: returns Iterator over all Bayesian Networks with one additional edge compared to `bn`
- `link_deletion(data: np.array, bn: BayesNet) -> Iterator[BayesNet]`: search operator: returns Iterator over all Bayesian Networks with one edge removed compared to `bn`
- `link_reversal(data: np.array, bn: BayesNet) -> Iterator[BayesNet]`: search operator: returns Iterator over all Bayesian Networks with one edge reversed compared to `bn`
- `greedy_local_search(bayes_net: BayesNet, data: np.array, score_fun=bic_score, successor_fun=successor_impl) -> BayesNet`: heuristic search procedure: applies search operators (`successor_fun`) to navigate through search space; uses heuristic function `score_fun` to rank Bayesian Networks resulting from search operators

We will implement a straightforward version of the heuristic search in the full space, without applying any strategies to enhance its efficiency. However, in the theoretical questions at the end of the assignment, you will be asked to outline ideas for improving the efficiency of the search.

<div class="alert alert-warning">
Implement <i>is_acyclic</i>, a function that determines whether a Bayesian Network is acyclic or not. (2 points)
</div>

This function will be needed in the next step for the search operators to check whether a specific search operator returns a valid Bayesian Network or not.

`is_acyclic` takes one parameter:
- `bn` is a Bayesian Network.

`is_acyclic` must return a boolean:
- True, if the Bayesian Network is acyclic, False otherwise.

*Hints:*
- Revisit your implementation of the topological sort. You may want to exploit the fact that a topological ordering can be found if and only if a directed graph has no directed cycles.
- `variable.children`: using the `children` attribute of `Variable` objects allows you to access a set of variable IDs representing a variable's child nodes.
- Remember that python allows you to define **inner** or **nested** functions in case you want to solve the task using recursion.

In [ ]:
def is_acyclic(bn: BayesNet) -> bool:
    """
    Checks whether Bayesian Network is acyclic or not.

    :param bn: A Bayesian Network object.
    :return: True, if the Bayesian Network is acyclic, False otherwise.
    """
    # all variables in the network encoded as a dictionary (keys: variable ids, values: Variable objects)
    variables = bn.nodes
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # no cycles have been found
    return True

In [ ]:
_A_, _B_, _C_ = 0, 1, 2
pdts = [
    (np.ones((2,))*0.5, (_A_,)), 
    (np.ones((2,2))*0.5, (_B_, _A_)), 
    (np.ones((2,2,2))*0.5, (_C_, _A_, _B_))
]
bn = BayesNet(*pdts)

assert isinstance(is_acyclic(bn), bool), "is_acyclic must return one boolean." 
assert is_acyclic(bn), f"Bayesian Network with index tables {[bn.indices]} is acyclic."

_A_, _B_, _C_ = 0, 1, 2
pdts = [
    (np.ones((2,2))*0.5, (_A_,_C_)), 
    (np.ones((2,2))*0.5, (_B_, _A_)), 
    (np.ones((2,2,2))*0.5, (_C_, _A_, _B_))
]
bn = BayesNet(*pdts)

assert isinstance(is_acyclic(bn), bool), "is_acyclic must return one boolean." 
assert not is_acyclic(bn), f"Bayesian Network with index tables {[bn.indices]} is cyclic."


<div class="alert alert-warning">
Implement <i>link_addition</i>, a function that returns an iterator over all valid Bayesian Networks with one additional edge added. (2 points)
</div>

`link_addition` is the first search operator you are going to implement.

`link_addition` takes two parameters:
- `data` is a NumPy array of shape `(num_samples, num_variables)`.
- `bn` is a Bayesian Network. *Assume that the parameters of this Bayesian Network are set to the maximum likelihood estimate w.r.t. to the data*.

`link_addition` must return one object:
- An iterator over all possible Bayesian Networks with one additional edge compared to the original Bayesian Network specified in the parameter `bn`. *The parameters of all returned Bayesian Networks must be set to the maximum likelihood estimate w.r.t. to the data*.

*Hints:*
- Think thoroughly about all conditions that need to be fulfilled in order to add an additional edge. For instance, don't forget to make use of your `is_acyclic` function.
- Mind that Python objects, such as lists, are passed by reference; updating a value in the list will affect all objects holding the reference! To avoid unintended change, create a copy of the list before you modify it:
```python
bn_pdts.copy()
```
- Remember that a Bayesian network object can be created from a list of pdts and indices in the following way:
```python
candidate = BayesNet(*zip(candidate_pdts, candidate_indices))
```

In [ ]:
def link_addition(data: np.array, bn: BayesNet) -> Iterator[BayesNet]:
    bn_pdts = list(bn.pdts)
    bn_indices = list(bn.indices)
    all_var_ids = bn.nodes.keys()
    all_vars = bn.nodes
    
    candidates = []
    # YOUR CODE HERE
    raise NotImplementedError()
    
    for candidate in candidates:
        yield candidate

In [ ]:
_A_, _B_ = 0, 1
data = np.array([[0, 0], [1, 1], [0, 1], [0, 0], [0, 1]])
pdts = [
    (maximum_likelihood_estimate(data, _A_), (_A_,)), 
    (maximum_likelihood_estimate(data, _B_), (_B_,)),
]
bn = BayesNet(*pdts)


actual = [b for b in link_addition(data, bn)]

expected = {
    "B->A": BayesNet(
        (maximum_likelihood_estimate(data, _A_, (_B_,)), (_A_,_B_)),
        (maximum_likelihood_estimate(data, _B_), (_B_,)),
    ),
    "A->B": BayesNet(
        (maximum_likelihood_estimate(data, _A_), (_A_,)),
        (maximum_likelihood_estimate(data, _B_, (_A_,)), (_B_,_A_)),
    ),
}

assert all([isinstance(b, BayesNet) for b in actual]), "Your function must only return BayesNet objects!"
for bn_expected in expected:
    in_actual = False
    for bn_actual in actual:
        if compare_bayes_net(expected[bn_expected], bn_actual, verbose=False):
            in_actual = True
    assert in_actual, f"The Bayesian Network with the additional link {bn_expected} is not returned by your function or the pdts do not match"
    
assert len(actual) == len(expected), "Number of Bayesian Networks differs from expected." \
    "If you pass the previous test, this means your implementation returns too many instances."


<div class="alert alert-warning">
Implement <i>link_deletion</i>, a function that returns an iterator over all Bayesian Networks with one edge removed. (2 points)
</div>

`link_deletion` is the second search operator you are going to implement.

`link_deletion` takes two parameters:
- `data` is a NumPy array of shape `(num_samples, num_variables)`.
- `bn` is a Bayesian Network. *Assume that the parameters of this Bayesian Network are set to the maximum likelihood estimate w.r.t. to the data*.

`link_deletion` must return one object:
- An iterator over all possible Bayesian Networks with one edge removed compared to the original Bayesian Network specified in the parameter `bn`. *The parameters of all returned Bayesian Networks must be set to the maximum likelihood estimate w.r.t. to the data*.

*Hints:*
- Mind that Python objects, such as lists, are passed by reference; updating a value in the list will affect all objects holding the reference! To avoid unintended change, create a copy of the list before you modify it:
```python
bn_pdts.copy()
```
- Remember that a Bayesian network object can be created from a list of pdts and indices in the following way:
```python
candidate = BayesNet(*zip(candidate_pdts, candidate_indices))
```

In [ ]:
def link_deletion(data: np.array, bn: BayesNet) -> Iterator[BayesNet]:
    bn_pdts = list(bn.pdts)
    bn_indices = list(bn.indices)
    
    candidates = []
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    for candidate in candidates:
        yield candidate

In [ ]:
_A_, _B_ = 0, 1
data = np.array([[0, 0], [1, 1], [0, 1], [0, 0], [0, 1]])
pdts = [
    (maximum_likelihood_estimate(data, _A_, (_B_,)), (_A_,_B_)), 
    (maximum_likelihood_estimate(data, _B_), (_B_,)),
]
bn = BayesNet(*pdts)


actual = [b for b in link_deletion(data, bn)]

expected = {
    "B->A": BayesNet(
        (maximum_likelihood_estimate(data, _A_), (_A_,)),
        (maximum_likelihood_estimate(data, _B_), (_B_,)),
    )
}

assert all([isinstance(b, BayesNet) for b in actual]), "Your function must only return BayesNet objects!"
for bn_expected in expected:
    in_actual = False
    for bn_actual in actual:
        if compare_bayes_net(expected[bn_expected], bn_actual, verbose=False):
            in_actual = True
    assert in_actual, f"The Bayesian Network with the removed link {bn_expected} not returned by your function or the pdts do not match"
assert len(actual) == len(expected), "Number of Bayesian Networks differs from expected." \
    "If you pass the previous test, this means your implementation returns too many instances."    


<div class="alert alert-warning">
Implement <i>link_reversal</i>, a function that returns an iterator over all valid Bayesian Networks with one edge reversed. (2 points)
</div>

`link_reversal` is the third and last search operator you are going to implement.

`link_reversal` takes two parameters:
- `data` is a NumPy array of shape `(num_samples, num_variables)`.
- `bn` is a Bayesian Network. *Assume that the parameters of this Bayesian Network are set to the maximum likelihood estimate w.r.t. to the data*.

`link_reversal` must return one object:
- An iterator over all possible Bayesian Networks with one edge reversed compared to the original Bayesian Network specified in the parameter `bn`. *The parameters of all returned Bayesian Networks must be set to the maximum likelihood estimate w.r.t. to the data*.

*Hints:*
- Think thoroughly about all conditions that need to be fulfilled in order to be able to reverse an edge. For instance, don't forget to make use of your `is_acyclic` function.
- Mind that Python objects, such as lists, are passed by reference; updating a value in the list will affect all objects holding the reference! To avoid unintended change, create a copy of the list before you modify it:
```python
bn_pdts.copy()
```
- Remember that a Bayesian network object can be created from a list of pdts and indices in the following way:
```python
candidate = BayesNet(*zip(candidate_pdts, candidate_indices))
```

In [ ]:
def link_reversal(data: np.array, bn: BayesNet) -> Iterator[BayesNet]:
    bn_pdts = list(bn.pdts)
    bn_indices = list(bn.indices)
    all_var_ids = bn.nodes.keys()
    all_vars = bn.nodes
    
    candidates = []
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    for candidate in candidates:
        yield candidate

In [ ]:
_A_, _B_ = 0, 1
data = np.array([[0, 0], [1, 1], [0, 1], [0, 0], [0, 1]])
pdts = [
    (maximum_likelihood_estimate(data, _A_, (_B_,)), (_A_,_B_)), 
    (maximum_likelihood_estimate(data, _B_), (_B_,)),
]
bn = BayesNet(*pdts)


actual = [b for b in link_reversal(data, bn)]

expected = {
    "B->A": BayesNet(
        (maximum_likelihood_estimate(data, _A_), (_A_,)),
        (maximum_likelihood_estimate(data, _B_, (_A_,)), (_B_,_A_)),
    )
}

assert all([isinstance(b, BayesNet) for b in actual]), "Your function must only return BayesNet objects!"
for bn_expected in expected:
    in_actual = False
    for bn_actual in actual:
        if compare_bayes_net(expected[bn_expected], bn_actual, verbose=False):
            in_actual = True
    assert in_actual, f"The Bayesian Network with the edge {bn_expected} reversed is not returned by your function or the pdts do not match"
assert len(actual) == len(expected), "Number of Bayesian Networks differs from expected." \
    "If you pass the previous test, this means your implementation returns too many instances."


In the following code cell, we chain the iterators returned from your implemented search operators `link_addition`, `link_deletion`, `link_reversal`. This allows us to conveniently call `successor_impl(data, bayes_net)`, and we will get all possible successors of the *bayes_net* object according to all search operators we implemented. Furthermore, we sort the Bayesian Networks returned from the search operators, to make your implementation of `greedy_local_search` comparable to ours.

**Nothing to do for you in the code cell below!**

In [ ]:
from itertools import chain

def successors(data: np.array, bn: BayesNet, operators: List[Iterator[BayesNet]]):
    successors = []
    for op in operators:
        successors.extend([next_bn for next_bn in op(data, bn)])
    
    # nothing to do for you here; this is just to make the ordering of successors the same for 
    # all implementations - we need this to test yours properly
    successors = sorted(successors, key=lambda s: str(s.structure_descriptor()))
    
    for s in successors:
        yield s
            
successor_impl = lambda data, bn: successors(data, bn, [link_addition, link_deletion, link_reversal])

In [ ]:
# leave this cell untouched, we just test the chaining of search operators here


### Greedy Best-First Search

Finally, we will implement a very simple greedy best-first search procedure that uses our search operators (`successor_impl`) and a scoring function (`score_fun` - e.g., the BIC score) to navigate through the search space.

<div class="alert alert-warning">
Implement <i>greedy_local_search</i>, the heuristic search procedure described in the lecture slides (slide deck 5b pp. 62). (2 points)
</div>

`greedy_local_search` takes four parameters:
- `data` is a NumPy array of shape `(num_samples, num_variables)`.
- `bayes_net` is the Bayesian Network the search starts from. *Assume that the parameters of this Bayesian Network are set to the maximum likelihood estimate w.r.t. to the data*.
- `score_fun` is the function used to score the candidate graph structure — defaults to `bic_score`. A higher score means a better model.
- `successor_fun` is the function used to generate the set of successors of a Bayesian Network. Can be called using: `successor_fun(data, bayes_net)` — defaults to `successor_impl`, the function that chains your implemented search operators. 

`greedy_local_search` must return one object:
- a BayesNet object that has the highest score (according to `score_fun`) of all Bayesian Networks visited during the greedy best-first search procedure.

**IMPORTANT:** In case there are multiple successors leading to the same best score, choose the first one returned from `successor_fun`. The test cases might fail if you don't do so!

*Hints:*
- Use `score_fun` to compute the score. Do not directly call your implementation of `bic_score`.
- Use `successor_fun(data, bayes_net)` to get all successors of `bayes_net`. Remember that your search operators are already implemented in a way such that they return all successors of a Bayesian network with their parameters set to the maximum likelihood estimate w.r.t. `data`.
- Use the BayesNet object `bayes_net` as the starting point for your search.
- At each step, greedily choose the Bayesian Network that gives the highest score and continue the search from this best-scoring Bayesian Network.
- Stop the search process when none of the successors has a higher score than your current Bayesian Network.

In [ ]:
def greedy_local_search(
    data: np.array,
    bayes_net: BayesNet,
    score_fun: Callable = bic_score,
    successor_fun: Callable = successor_impl
) -> BayesNet:
    """
    Perform greedy local search to optimize the structure of a Bayesian Network.

    Parameters:
    - data (np.array): Data used for scoring the Bayesian networks.
    - bayes_net (BayesNet): The initial Bayesian Network to start the search.
    - score_fun (Callable): A function to compute the score of a network. Higher is better.
    - successor_fun (Callable): A function to generate successors of a network.

    Returns:
    - BayesNet: The Bayesian Network with the highest score found during the search.
    """
    current_bn = bayes_net
    current_score = score_fun(data, current_bn)
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return current_bn

In the following code cell, we test your algorithm by creating the following Bayesian Network with three variables A, B and C, the edges A->B, A->C, and B->C, and the given probability distribution tables. We generate 6000 samples from the created Bayesian Network and then try to rediscover its structure with your implementation of `greedy_local_search`. We pass the generated `data`, the unconnected network `b0`, the `bic_score`, and the successor function `successor_impl` to `greedy_local_search`. We plot the Bayesian Network found by `greedy_local_search`.  
Can you recover the original network structure that was used to generate the data? If yes, your implementation is most likely correct! 
<img  width='30%' src='img/simple_bn.svg'>  
 
The conditional probability tables are given as:

<table style="float: left;margin:5px;"><tr><th>P(A)</th><th>$a_0$<br></th><th>$a_1$</th></tr><tr><td>-</td><td>0.1</td><td>0.9</td></tr></table>

<table style="float: left;margin:5px;"><tr><th>P(B | A)</th><th>$a_0$<br></th><th>$a_1$</th></tr><tr><td>$b_0$</td><td>0.25</td><td>0.8</td></tr><tr><td>$b_1$</td><td>0.75</td><td>0.2</td></tr></table>

<table style="float: left;margin:5px;"><tr><th rowspan="2">P(C | A, B)</th><th colspan="2">$a_0$<br></th><th colspan="2">$a_1$</th></tr><tr><td>$b_0$</td><td>$b_1$</td><td>$b_0$</td><td>$b_1$</td></tr><tr><td>$c_0$<br></td><td>0.99</td><td>0.5</td><td>0.9</td><td>0.05</td></tr><tr><td>$c_1$</td><td>0.01</td><td>0.5</td><td>0.1</td><td>0.95</td></tr></table>    


In [ ]:
# create true underlying bayes net
_A_, _B_, _C_ = 0, 1, 2

A = np.array([0.1, 0.9])
B_A = np.array([[0.25, 0.8], [0.75, 0.2]])
C_AB = np.array([[[0.99, 0.5], [0.9, 0.05]], [[0.01, 0.5], [0.1, 0.95]]])

pdts = [
    (A, (_A_,)), 
    (B_A, (_B_, _A_)),
    (C_AB, (_C_, _A_, _B_)),
]

true_bn = BayesNet(*pdts)

# generate data
np.random.seed(0)
data = sample_forward(true_bn, 6000)

# unconnected bayes net to start search from
b0 = BayesNet(
    (maximum_likelihood_estimate(data, _A_), (_A_,)), 
    (maximum_likelihood_estimate(data, _B_), (_B_,)),
    (maximum_likelihood_estimate(data, _C_), (_C_,)),
)
    
best_bn = greedy_local_search(data, b0, score_fun=bic_score, successor_fun=successor_impl)

assert isinstance(best_bn, BayesNet), "'greedy_local_search' must return an instance of a Bayesian Network."

draw_graph(
    best_bn, 
    node_names=['A', 'B', 'C'], 
    pos={'A': [0., 1.], 'B': [1., 1.], 'C': [.5, .5]}
)


## Back to Strokes

---


Equipped with the freshly acquired knowledge about structure learning, you revisit the probabilistic medical world of Problem Set 3, which comprises the following random variables:

 - A ... Alcoholism
 - H ... High Blood Pressure
 - S ... Stroke
 - C ... Confusion
 - V ... Vertigo

This time, you aim to compensate for your lack of domain knowledge by taking a more data-driven approach, letting the data guide you in finding the correct structure.

Below is the true underlying Bayesian model (structure and parameters) used to generate the training and test data for you.

All variables in this example are Boolean (false = 0, true = 1).

<img  style='width:100%;  max-width:400px;' src="img/bn_mod2.svg">

The conditional probability tables are given as follows:

<table style="float: left;margin:5px;"><tr><th>P(A)</th><th>$a_0$<br></th><th>$a_1$</th></tr><tr><td>-</td><td>0.01</td><td>0.99</td></tr></table>

<table style="float: left;margin:5px;"><tr><th>P(H | A)</th><th>$a_0$<br></th><th>$a_1$</th></tr><tr><td>$h_0$</td><td>0.9</td><td>0.8</td></tr><tr><td>$h_1$</td><td>0.1</td><td>0.2</td></tr></table>

<table style="float: left;margin:5px;"><tr><th>P(S | H)</th><th>$h_0$<br></th><th>$h_1$</th></tr><tr><td>$s_0$</td><td>0.9</td><td>0.85</td></tr><tr><td>$s_1$</td><td>0.1</td><td>0.15</td></tr></table>


<table style="float: left;margin:5px;"><tr><th rowspan="2">P(C | A, S)</th><th colspan="2">$a_0$<br></th><th colspan="2">$a_1$</th></tr><tr><td>$s_0$</td><td>$s_1$</td><td>$s_0$</td><td>$s_1$</td></tr><tr><td>$c_0$<br></td><td>0.8</td><td>0.7</td><td>0.85</td><td>0.45</td></tr><tr><td>$c_1$</td><td>0.2</td><td>0.3</td><td>0.15</td><td>0.55</td></tr></table>

<table style="float: left;margin:5px;"><tr><th>P(V | S)</th><th>$s_0$</th><th>$s_1$</th></tr><tr><td>$v_0$</td><td>0.1</td><td>0.2</td></tr><tr><td>$v_1$</td><td>0.9</td><td>0.8</td></tr></table>  

We generate three datasets of varying sizes (1,000, 10,000, and 20,000 data points) along with a test set (10,000 data points). These datasets are stored in the files *train_small*, *train_medium*, *train_big*, *test*.

Now, it is your turn to (re-)discover the true underlying Bayesian network that generated the data.

In [ ]:
_A_, _H_, _S_, _C_, _V_ = 0, 1, 2, 3, 4

train_small = np.loadtxt('data/train_small.txt', dtype=np.int64)
train_medium = np.loadtxt('data/train_medium.txt', dtype=np.int64)
train_big = np.loadtxt('data/train_big.txt', dtype=np.int64)
test = np.loadtxt('data/test.txt', dtype=np.int64)

Although you have a hard time determining the dependencies among the random variables directly, you have some idea which of the variables might be dependent on each other and, therefore, you decide to use **Complete Search in Restricted Space** to find the best model. You come up with several *variable orders* to restrict the search space strongly and consider the following configurations:
<table style="border-collapse:collapse;border-spacing:0;border-color:#ccc"><tr><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:bold;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#f0f0f0">Variable Order</th><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:bold;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#f0f0f0">Score Function</th></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff">A, H, S, C, V</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff">Bayesian Information Criterion</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff">V, C, S, H, A</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff">Bayesian Information Criterion</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff;vertical-align:top">C, A, H, V, S<br></td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff;vertical-align:top">Bayesian Information Criterion</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff;vertical-align:top">A, H, S, C, V</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff;vertical-align:top">Log-Likelihood</td></tr></table>

<div class="alert alert-warning">
    
Use the `complete_search` function to learn a graph structure and the parameters for the different configurations and for all data sets (`train_small`, `train_medium`, and `train_big`). Store the bayes_net objects into the provided result variable `cs_models` and visualize the learned graph structures with `draw_graph(bayes_net)`. Use this section to analyze the results more deeply! (2 points)
</div>

*Hints:*
- You can use `log_likelihood` as a scoring function by overwriting the default choice for the named parameter `score_fun` of `complete_search`, e.g.,
```python
complete_search(data, order, score_fun=log_likelihood)
```

In [ ]:
help(draw_graph)
help(evaluate_bayes_net)

In [ ]:
### this is part of a hidden test; don't remove...

In [ ]:
cs_models = {}
for data_set_size in ['small', 'medium', 'big']:
    cs_models[data_set_size] = {
            'ordered': None, # _A_, _H_, _S_, _C_, _V_; bic_score
            'reverse': None, # _V_, _C_, _S_, _H_, _A_; bic_score
            'random': None, # _C_, _A_, _H_, _V_, _S_; bic_score
            'log-likelihood': None, # _A_, _H_, _S_, _C_, _V_; log_likelihood
        }

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
for data_set_id in ['small', 'medium', 'big']:
    for combination_id in ['ordered', 'reverse', 'random', 'log-likelihood']:
        assert type(cs_models[data_set_id][combination_id]) is BayesNet
        

Given all these results, you still feel uncertain about the learned Bayesian Network structures and begin to question your understanding of medicine. Were the specified variable orderings adequate? Then a thought strikes you: why not try heuristic search in the full space, where specifying a variable order is not required?

You turn to **Heuristic Search in Full Space** and consider the following configurations:

<table style="border-collapse:collapse;border-spacing:0;border-color:#ccc"><tr><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:bold;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#f0f0f0">Starting Structure</th><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:bold;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#f0f0f0">Score Function</th></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff">Unconnected</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff">Bayesian Information Criterion</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff">Result Complete Search: log-likelihood</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff">Bayesian Information Criterion</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff;vertical-align:top">Result Complete Search: ordered<br></td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff;vertical-align:top">Bayesian Information Criterion</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff;vertical-align:top">Unconnected</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff;vertical-align:top">Log-Likelihood</td></tr></table>

*Starting Structure* determines the structure of `bayes_net` that we pass to `greedy_local_search`. The two settings *Result Complete Search: log-likelihood* and *Result Complete Search: ordered* refer to the structures we obtained as the result of *Complete Search in Restricted Space* with the variable ordering *A, H, S, C, V* using the log-likelihood and the bic score as score function, respectively. 

<div class="alert alert-warning">
    
Use the `greedy_local_search` function to learn a graph structure and the parameters for the different configurations and for all data sets (`train_small`, `train_medium`, and `train_big`). Store the bayes_net objects into the provided result variable `hs_models` and visualize the learned graph structures with `draw_graph(bayes_net)`. Use this section to analyze the results more deeply! (2 points)
</div>

**Important**: For the starting structures obtained by *Complete Search in Restricted Space*, the dataset sizes must match. For instance, if you run the configuration with starting structure *Result Complete Search: ordered* on `train_medium`, you will use `cs_models['medium']['ordered']` as the starting structure passed to `greedy_local_search` (and likewise `cs_models['small']['ordered']` for `train_small` and `cs_models['large']['ordered']` for `train_large`).

*Hints:*
- You can use `log_likelihood` as a scoring function by overwriting the default choice for the named parameter `score_fun` of `greedy_local_search`, e.g.,
```python
greedy_local_search(data, order, score_fun=log_likelihood)
```

In [ ]:
### this is part of a hidden test; don't remove...

In [ ]:
hs_models = {}
for data_set_size in ['small', 'medium', 'big']:
    hs_models[data_set_size] = {
                    'unconnected': None,  # unconnected network; bic score
                    'cs_log-likelihood': None,  # network resulting from complete search ('log-likelihood'); bic score
                    'cs_ordered': None,  # network resulting from complete search ('ordered'); bic score
                    'log-likelihood': None  # unconnected; log-likelihood
            } 

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
for data_set_id in ['small', 'medium', 'big']:
    for combination_id in ['unconnected', 'cs_log-likelihood', 'cs_ordered', 'log-likelihood']:
        assert type(hs_models[data_set_id][combination_id]) is BayesNet
        

### Interpret the Results in Detail

In the following, we will compare all learned Bayesian Network structures (stored in variables `cs_models` and `hs_models`) w.r.t. their log-likelihoods on train and test datasets.

The first type of plots visualize the training set size's influence on the log-likelihood relative to the test data.

**Complete Search in Restricted Space:**

In [ ]:
from utils import plot_compare_orders, compare_train_size

compare_train_size(cs_models, [train_small, train_medium, train_big], test)

**Heuristic Search in Full Space**:

In [ ]:
compare_train_size(hs_models, [train_small, train_medium, train_big], test, cs=False)

The second type of plots compare all configurations' log-likelihood relative to the training and test data for each dataset size.

**Complete Search in Restricted Space:**

In [ ]:
plot_compare_orders(cs_models, [train_small, train_medium, train_big], test)

**Heuristic Search in Full Space**:

In [ ]:
plot_compare_orders(hs_models, [train_small, train_medium, train_big], test, cs=False)

<div class="alert alert-warning">
Consider the implemented functions and the resulting plots in more detail and answer the following questions *briefly*. (1 point each)
</div>

Looking at the plots and the test likelihood scores: (1) In this specific situation, which method is preferable: **Complete Search in Restricted Space** or **Heuristic Search in Full Space**? (2) In general, in which situations would you recommend to use **Complete Search in Restricted Space** and in which **Heuristic Search in Full Space**?

YOUR ANSWER HERE

Given a specific train dataset and the BIC score as the scoring function. Let's assume that we apply **Complete Search in Restricted Space** with the correct ordering of variables (i.e., one that corresponds to a topological ordering of the variables in the Bayesian Network representing the true underlying full joint distribution). Let's further assume that **Complete Search in Restricted Space** returns a good but suboptimal structure (one that is different from the true structure). Can we discover the true underlying structure by passing the suboptimal structure as the starting structure to our **Heuristic Search in Full Space** algorithm?

Store `True` in the provided variable `results` if the true underlying structure can be discovered by **Heuristic Search in Full Space** and `False` otherwise.

In [ ]:
result = None  # store True or False into this variable

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert result is not None
assert result in [True, False]

Our implementation of **Heuristic Search in Full Space** is functionally correct but suffers from inefficiency due to redundant computations. Can you identify where these redundant computations occur in our code and suggest two ideas we could leverage to improve its efficiency?

YOUR ANSWER HERE

In **Complete Search in Restricted Space**, the best parent assignment for any of the earlier variables might change depending on the parents chosen for any of the later variables. 

Is this statement correct? Store the correct answer (`True` or `False`) in the provided variable `results`.

In [ ]:
result = None  # store True or False into this variable

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert result is not None
assert result in [True, False]

Let $N \gt 3$ denote the number of random variables in a specific probabilistic model. Given two arbitrary Bayesian Networks $G_1$ and $G_2$ and the search operators we implemented in this problem set. Select the smallest number of search steps (one step = modifying one edge with one of our three implemented search operators) from the possible answers below that guarantees us to be able to traverse the search space from $G_1$ to $G_2$.

(1) $N$

(2) $N^2$

(3) $2^N$

(4) $N!$

(5) $N^N$

Store the correct answer 1-5 in the provided variable `results`.

In [ ]:
result = None

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert result is not None
assert result in [1,2,3,4,5]